In [349]:
#Skripte laden
%reload_ext autoreload
%autoreload 2
import os
os.chdir('/home/safiul/email-classification')
import data
from Email_Classification.Functions.packages import *
import Email_Classification.Functions.Functions as fn
import Email_Classification.Functions.Helper as hp
import Email_Classification.Functions.parameters as par

In [ ]:
%%time
with open(os.path.join(path_model, 'feature/X_test_vec.npy'), 'rb') as f:
    X_test_vec = np.load(f)
with open(os.path.join(path_model, 'feature/y_test.npy'), 'rb') as f:
    y_test = np.load(f)

In [561]:
#Ergebnistabelle initialisieren
result_mean = pd.DataFrame(columns=['n', 'Accuracy', 'AUROC Succ/Pred. Prob', 'AUROC Succ/Entropy',
       'AUROC/lr'])
result_mean = pd.DataFrame(columns=['n', 'Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy',
       'AUROC/lr'])
result_mode = pd.DataFrame(columns=['n', 'Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy',
       'AUROC/lr'])
result_random =pd.DataFrame(columns=['n', 'Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy',
       'AUROC/lr'])
result = pd.DataFrame(columns = ['Modelle', 'Accuracy', 'AUROC Succ/Pred. Prob', 'AUROC Succ/Entropy',
       'AUROC/lr'])

In [571]:
#Pfade + Netzwerkarchitektur festlegen
Model_name = 'CNN'
sample_num = 50
filter_size = 200
lstm_layer =  200
path_model = 'Email_Classification/Models/Sequential Models/\
rnn/lstm/Models/uncertainity/VI/phi_1_sig1_10_sig2_0.1'
path_result = 'Email_Classification/Models/Sequential Models/rnn/\
lstm/Models/uncertainity/VI/phi_1_sig1_10_sig2_0.1/result'
path_weight = os.path.join(path_model, 'weights.best.hdf5')
path_sample = os.path.join(path_result, 'MC_{}'.format(sample_num), 'sample.npy')
path_mc_mean = os.path.join(path_result, 'MC_{}'.format(sample_num), 'pred_prob_mean.npy')
path_mc_mode = os.path.join(path_result, 'MC_{}'.format(sample_num), 'pred_prob_mode.npy')

In [210]:
%%time
#Test- + Trainingsmenge und Modelle laden
with open(os.path.join(path_model, 'feature/X_test_vec.npy'), 'rb') as f:
    X_test_vec = np.load(f)
with open(os.path.join(path_model, 'feature/y_test.npy'), 'rb') as f:
    y_test = np.load(f)
rnn_model = tf.keras.models.load_model(path_model)
rnn_model.load_weights(path_weight)   

CPU times: user 632 ms, sys: 503 ms, total: 1.14 s
Wall time: 3.08 s


In [ ]:
#Erstellt Ergebnistabelle für deterministische Netzwerke
pred_prob = rnn_model.predict(X_test_vec)
test_label = y_test.argmax(axis = 1)
Accuracy, auroc_final , auroc_metaclf,_,_,_ = 
fn.result_final(pred_prob = pred_prob, test_label = test_label)
res = np.zeros(6)
#res = np.append([filter_size], lstm_layer)
res = np.append([Model_name], Accuracy)
res = np.append(res, auroc_final)
res = np.append(res, [auroc_metaclf])
res_series = pd.Series(res, index = result.columns)
result = result.append(res_series, ignore_index=True)
result.iloc[:, 1:5] = result.iloc[:, 1:5].apply(pd.to_numeric).round(4)
result.to_csv(path_or_buf =  os.path.join(path_result,'result.csv'), index = False)

In [517]:
%%time
#Monte-Carlo-Simulation durchführen und die Outputs speichern
pred_prob_mode, pred_prob_mean, sample = fn.run_mc(rnn_model, sample_num, X_test_vec, y_test)
#write data
with open(path_sample, 'wb') as f:
    np.save(f, sample)
with open(path_mc_mean, 'wb') as f:
    np.save(f, pred_prob_mean)
with open(path_mc_mode, 'wb') as f:
    np.save(f, pred_prob_mode)

CPU times: user 15h 51min 51s, sys: 9min 25s, total: 16h 1min 17s
Wall time: 1h 16min 27s


In [572]:
#Outputs der Monte-Carlo-Simulation laden
with open(path_sample, 'rb') as f:
    sample = np.load(f)
with open(path_mc_mean, 'rb') as f:
    pred_prob_mean = np.load(f)
with open(path_mc_mode, 'rb') as f:
    pred_prob_mode = np.load(f)

In [ ]:
%%time
#Erstellt Ergebnistabelle für probabilistische Netzwerke - 
#für den Modellschätzer (Alternativ I)
test_label = y_test.argmax(axis = 1)
Accuracy, auroc_final , auroc_metaclf,_,_,_ = \
fn.result_final(pred_prob = pred_prob_mean, y_test = y_test)
res = np.zeros(10)
res = np.append([sample_num], Accuracy)
res = np.append(res, auroc_final)
res = np.append(res, [auroc_metaclf])
res_series = pd.Series(res, index = result_mean.columns)
result_mean = result_mean.append(res_series, ignore_index=True).round(4)
result_mean.to_csv(path_or_buf =  os.path.join(path_model, 'result/result_mean.csv'), index = False)

In [ ]:
%%time
#Erstellt Ergebnistabelle für probabilistische Netzwerke - 
#für den Modellschätzer (Alternativ II)
test_label = y_test.argmax(axis = 1)
Accuracy, auroc_final , auroc_metaclf,_,_,_ = \
fn.result_final(pred_prob = pred_prob_mode, y_test = y_test)
res = np.zeros(5)
res = np.append([sample_num], Accuracy)
res = np.append(res, auroc_final)
res = np.append(res, [auroc_metaclf])
res_series = pd.Series(res, index = result_mode.columns)
result_mode = result_mode.append(res_series, ignore_index=True).round(4)
result_mode.to_csv(path_or_buf =  os.path.join(path_model, 'result/result_mode.csv'), index = False)

In [ ]:
#Ein Wörterbuch für Unsicherheitsmetriken erstellen
#df_final = dict()
dict_meta = { 'pred. prob.': feature_test_meta[:,0],
            'Entropy': -feature_test_meta[:,1],
            'lr': lr.predict_proba(feature_test_meta)[:,1],
            'labels': label_test_meta}
df_meta = pd.DataFrame(dict_meta)
df_meta['labels'] = df_meta['labels'].apply(lambda x: 'korrekt (Dropout)' if x == 1 else 'falsch (Dropout)')
df_final['Dropout'] = df_meta

In [ ]:
%%time
#Erstellt Ergebnistabelle für probabilistische Netzwerke - 
#für die anzahl der Stichprobe n = 1
rand_sam = np.random.choice(sample_num, 1)[0]
pred_prob_random = sample[rand_sam]
test_label = y_test.argmax(axis = 1)
Accuracy, auroc_final , auroc_metaclf,_,_,_ = \
fn.result_final(pred_prob = pred_prob_random, y_test = y_test)
res = np.zeros(5)
res = np.append([sample_num], Accuracy)
res = np.append(res, auroc_final)
res = np.append(res, [auroc_metaclf])
res_series = pd.Series(res, index = result_random.columns)
result_random = result_random.append(res_series, ignore_index=True).round(4)
result_random.to_csv(path_or_buf =  os.path.join(path_model,'result/result_random.csv'), index = False)

In [427]:
#Zusammenfügen aller Ergebnistabelle
result_random = pd.read_csv('Email_Classification/Models/Sequential Models/cnn/model/uncertainty\
/2020-12-01_0124_dropout_0.25_filter_200/result/result_random.csv')
result_mode = pd.read_csv('Email_Classification/Models/Sequential Models/cnn/model/\
uncertainty/2020-12-01_0124_dropout_0.25_filter_200/result/result_mode.csv')
result_mean = pd.read_csv('Email_Classification/Models/Sequential Models/cnn/model/\
uncertainty/2020-12-01_0124_dropout_0.25_filter_200/result/result_mean.csv')
result_full = pd.concat([result_random, result_mode, result_mean])
result_full['Model'] = ''
result_full = result_full[['Model', 'n', 'Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy', 'AUROC/lr']]
result_full.index = list(range(9))
result_full.loc[0:2, 'Model'] = 'MC Random'
result_full.loc[3:5, 'Model'] = 'MC Mode'
result_full.loc[6:8, 'Model'] = 'MC Meam'
result_full = result_full.sort_values(by = ['n'])
result_full.index = list(range(9))
result_full = result_full.round(4)
result_full.to_csv(path_or_buf =  os.path.join('Email_Classification/Models/\
Sequential Models/cnn/model/uncertainty/2020-12-01_0124_dropout_0.25_filter_200', 
                                               'result/result_full.csv'), index = False)

In [456]:
#Ergebnisse in einer tex Datei speichern
result = pd.read_csv('Email_Classification/Models/Sequential Models/rnn/bi-lstm/\
uncertainty/VI/2020-12-04_1952_filter_200_unit_200/result/result_full.csv')
#result = pd.read_csv('Email_Classification/Models/Sequential Models/result.csv')
for i in ['Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy','AUROC/lr']:
    result[i] = result[i].map(lambda n: '{:,.2%}'.format(n))
result.columns = ['Klassifikatoren','n', 'Genauigkeit', 'PkW', 'Entropie', 'lr']
result = result.drop(['n'], axis = 1)
result.to_latex(os.path.join('Email_Classification/Models/Sequential Models/rnn/bi-lstm/\
uncertainty/VI/2020-12-04_1952_filter_200_unit_200/result', 'tex/result_full.tex'), index = False)

In [ ]:
#Violin-Plot für die Unsicherheitsverteilungen
with open('Email_Classification/Models/Nonsequential Models/result/result_data/df_final.pkl', 'rb') as f:
    df_ = pickle.load(f)#matplotlib.rcParams.update({'font.size': 50})
sns.set(font_scale= 2.5)
sns.set_style("white")

plt.figure(figsize=(40,25))
plt.subplot(3, 4, 1)
fig1 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y =  df_final['LGBM']['pred. prob.'] ,
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 2)
fig1 = sns.violinplot(x = df_final['C-LSTM']['labels'],  
              y =  df_final['C-LSTM']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=45)

plt.subplot(3, 4, 3)
fig1 = sns.violinplot(x = df_final['VB']['labels'],  
              y =  df_final['VB']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 4)
fig1 = sns.violinplot(x = df_final['Dropout']['labels'],  
              y =  df_final['Dropout']['pred. prob.'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('WpK')
#plt.setp(fig1.get_yticklabels(), rotation=90)



plt.subplot(3, 4, 5)
fig2 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y = df_final['LGBM']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 6)
fig2 = sns.violinplot(x = df_final['C-LSTM']['labels'],  
              y = df_final['C-LSTM']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 7)
fig2 = sns.violinplot(x = df_final['VB']['labels'],  
              y = df_final['VB']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 8)
fig2 = sns.violinplot(x = df_final['Dropout']['labels'],  
              y = df_final['Dropout']['Entropy'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('Entropie')
#plt.setp(fig2.get_yticklabels(), rotation=90)


plt.subplot(3, 4, 9)
fig3 = sns.violinplot(x = df_final['LGBM']['labels'],  
              y =  df_final['LGBM']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 10)
fig3 = sns.violinplot(x = df_final['C-LSTM']['labels'],  
              y =  df_final['C-LSTM']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 11)
fig3 = sns.violinplot(x = df_final['VB']['labels'],  
              y =  df_final['VB']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.subplot(3, 4, 12)
fig3 = sns.violinplot(x = df_final['Dropout']['labels'],  
              y =  df_final['Dropout']['lr'],
              scale="width",
               color = "lightgray"
              )
plt.xlabel('')
plt.ylabel('LR (Metaklassif.)')
#plt.setp(fig3.get_yticklabels(), rotation=90)

plt.savefig('Email_Classification/Models/Sequential Models/result/\
figure/dist_meta_classif_final.pdf', bbox_inches = 'tight',
    pad_inches = 0.1)
plt.show()

In [439]:
#exportieren Ergebnisse als tex Datei
c_lstm = pd.read_csv('Email_Classification/Models/Sequential Models/rnn/\
lstm/Models/uncertainity/McDropout/2020-11-10_1002_dropout_conv0.25_\
recurrent_0.25_nofinallayerdropout/result/result_full.csv')
bi_lstm = pd.read_csv('Email_Classification/Models/Sequential Models/rnn/\
bi-lstm/uncertainty/MCDropout/2020-11-10_1002_dropout_conv0.25_recurrent_nofinallayer/result/result_full.csv')
cnn = pd.read_csv('Email_Classification/Models/Sequential Models/\
cnn/model/uncertainty/2020-12-01_0124_dropout_0.25_filter_200/result/result_full.csv')
c_lstm = c_lstm[7:9]
bi_lstm = bi_lstm[7:9]
cnn = cnn[7:9]
df = cnn.append([c_lstm, bi_lstm])
df = df.drop(['n'], axis = 1)
for i in ['Accuracy', 'AUROC/Pred. Prob', 'AUROC/Entropy','AUROC/lr']:
    df[i] = df[i].map(lambda n: '{:,.2%}'.format(n))
df.columns = ['Netzwerkarchitekturen','Genauigkeit', 'PkW', 'Entropie', 'lr']
df.to_latex(os.path.join(path_result, 'tex/result_summary.tex'), index = False)

In [499]:
#Kolmogorov-smrinov Test
model = 'Dropout'
#test_final = pd.DataFrame(columns = ['pred. prob.', 'Entropy', 'lr'])
df1 = df_final[model][df_final[model].labels == 'korrekt ({})'.format(model)]
df2 = df_final[model][df_final[model].labels != 'korrekt ({})'.format(model)]
dict_test = { 'pred. prob.': [scipy.stats.ks_2samp(df1['pred. prob.'], df2['pred. prob.']).statistic,
                              scipy.stats.ks_2samp(df1['pred. prob.'], df2['pred. prob.']).pvalue],
            'Entropy': [scipy.stats.ks_2samp(df1['Entropy'], df2['Entropy']).statistic,
                              scipy.stats.ks_2samp(df1['Entropy'], df2['Entropy']).pvalue],
            'lr': [scipy.stats.ks_2samp(df1['lr'], df2['lr']).statistic,
                              scipy.stats.ks_2samp(df1['lr'], df2['lr']).pvalue]
            }
df_test = pd.DataFrame(dict_test)
test_final = test_final.append(df_test)
test_final.to_latex('Email_Classification/Models/Sequential Models/result/result_test.tex')
#Kritischer Wert für KS-Test berechnen
criticalvalue_KS(alpha = 0.01, n = df1, m = df2).round(4)

In [ ]:
#Tabelle der Ergebnisse aller gefitteten Modelle
#plot_data = pd.DataFrame(columns = ['Model','Accuracy', 'WpK', 'Entropie', 'LR'])
dat = pd.read_csv('Email_Classification/Models/Sequential Models/cnn/model/single_model/result.csv')
dat = dat.drop(['n_feature'], axis = 1)
dat['model'] = 'VB'
dat.columns = ['Accuracy', 'WpK', 'Entropie', 'LR', 'Model']
dat = dat[['Model','Accuracy', 'WpK', 'Entropie', 'LR']]
plot_data = plot_data.append(dat)

In [ ]:
#grafische Darstellung der Genauigkeit und der Unsicherheit
plot_data = pd.read_csv('Email_Classification/Models/Sequential Models/plot_data.csv')
res2 = plot_data.loc[plot_data.Model.isin(['Plain-Net', 'MCD', 'VB'])]
res1 = plot_data.loc[~plot_data.Model.isin(['Plain-Net', 'MCD', 'VB'])]

from matplotlib.ticker import FormatStrFormatter
sns.set(font_scale= 1.9)
sns.set_style("white")
matplotlib.rcParams.update({'font.size': 20})
plt.rc('axes', labelsize=25) 
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(27, 18))
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax4.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax5.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax6.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax1.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax2.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax3.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax4.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax5.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax6.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

groups = res1.groupby('Model')
markers =  ['*', 'o', 'v', '^']
markerfacecolor = ['blue', 'green', 'red',  'magenta']
s = 20
i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['WpK']
    ax1.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res1['Accuracy'], res1['WpK'], 1)
    ax1.plot(res1['Accuracy'], m*res1['Accuracy'] + b, color = 'black')
    ax1.legend(loc = 'lower right', fontsize = 'medium')
    ax1.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (WpK)')


i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['Entropie']
    ax2.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res1['Accuracy'], res1['Entropie'], 1)
    ax2.plot(res1['Accuracy'], m*res1['Accuracy'] + b, color = 'black')
    ax2.legend(loc = 'lower right', fontsize = 'medium')
    ax2.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (Entropie)')
i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['LR']
    ax3.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res1['Accuracy'], res1['LR'], 1)
    ax3.plot(res1['Accuracy'], m*res1['Accuracy'] + b, color = 'black')
    ax3.legend(loc = 'lower right', fontsize = 'medium')
    ax3.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (LR)')
    
    
    
groups = res2.groupby('Model')
markers =  ['*', 'o', 'v']
markerfacecolor = ['blue', 'green', 'red']
s = 20
i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['WpK']
    ax4.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res2['Accuracy'], res2['WpK'], 1)
    ax4.plot(res2['Accuracy'], m*res2['Accuracy'] + b, color = 'black')
    ax4.legend(loc = 'lower right', fontsize = 'medium')
    ax4.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (WpK)')


i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['Entropie']
    ax5.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res2['Accuracy'], res2['Entropie'], 1)
    ax5.plot(res2['Accuracy'], m*res2['Accuracy'] + b, color = 'black')
    ax5.legend(loc = 'lower right', fontsize = 'medium')
    ax5.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (Entropie)')
i = 0
for name, group in groups:
    mi = markers[i]
    mf = markerfacecolor[i]
    i = i +1
    x = group['Accuracy']
    y = group['LR']
    ax6.plot(x,y, marker = mi, markerfacecolor = mf, linestyle = "", label = name, markersize= s)
    m, b = np.polyfit(res2['Accuracy'], res2['LR'], 1)
    ax6.plot(res2['Accuracy'], m*res2['Accuracy'] + b, color = 'black')
    ax6.legend(loc = 'lower right', fontsize = 'medium')
    ax6.set(xlabel = 'Genauigkeit', ylabel = 'AUROC (LR)')
    
plt.savefig('Email_Classification/Models/Sequential Models/result/auroc_vs_acc.pdf', bbox_inches = 'tight',
   pad_inches = 0.1)
plt.show()  